## Jak2/STAT5 Model

The goal of the notebook is to study practical identifiability of **JAK2/STAT5 Model** with *CICOBase*. Identifiability of this model was analyzed in [Bachmann et al. (2011), Mol Syst Biol. ; 7: 516.](https://pubmed.ncbi.nlm.nih.gov/21772264/). We have translated the original Matlab model from [Bachmann_MSB2011 repo](https://github.com/Data2Dynamics/d2d/tree/master/arFramework3/Examples/Bachmann_MSB2011) to [Julia language](https://julialang.org/). 
The model is defined by the system of ODE

### Initial Value Problem

In [1]:
using DiffEqBase, DiffEqCallbacks, CSV, DataFrames, Sundials, RecursiveArrayTools

#compartments
const cyt = 0.4
const nuc = 0.275

function ode_func(du, u, p, t, epo_level, ActD, CISoe, SOCS3oe)
    # 25 states:
    (EpoRJAK2,
    EpoRpJAK2,
    p1EpoRpJAK2,
    p2EpoRpJAK2,
    p12EpoRpJAK2,
    EpoRJAK2_CIS,
    SHP1,
    SHP1Act,
    STAT5,
    pSTAT5,
    npSTAT5,
    CISnRNA1,
    CISnRNA2,
    CISnRNA3,
    CISnRNA4,
    CISnRNA5,
    CISRNA,
    CIS,
    SOCS3nRNA1,
    SOCS3nRNA2,
    SOCS3nRNA3,
    SOCS3nRNA4,
    SOCS3nRNA5,
    SOCS3RNA,
    SOCS3) = u
    
    # 26 parameters + 3 are initial conditions 
    (CISEqc,
    CISEqcOE,
    CISInh,
    CISRNADelay,
    CISRNAEqc,
    CISRNATurn,
    CISTurn,
    EpoRActJAK2,
    EpoRCISInh,
    EpoRCISRemove,
    JAK2ActEpo,
    JAK2EpoRDeaSHP1,
    SHP1ActEpoR,
    SHP1Dea,
    SHP1ProOE,
    SOCS3Eqc,
    SOCS3EqcOE,
    SOCS3Inh,
    SOCS3RNADelay,
    SOCS3RNAEqc,
    SOCS3RNATurn,
    SOCS3Turn,
    STAT5ActEpoR,
    STAT5ActJAK2,
    STAT5Exp,
    STAT5Imp,
    init_EpoRJAK2,
    init_SHP1,
    init_STAT5) = p
    
    # treatment
    Epo = epo_level
    
    # rates
    v1 = (Epo*EpoRJAK2*JAK2ActEpo)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1)
    v2 = EpoRpJAK2*(JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act
    v3 = (EpoRpJAK2*EpoRActJAK2)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1)
    v4 = (3*EpoRpJAK2*EpoRActJAK2)/((EpoRCISInh*EpoRJAK2_CIS + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1))
    v5 = (3*EpoRActJAK2*p1EpoRpJAK2)/((EpoRCISInh*EpoRJAK2_CIS + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1))
    v6 = (EpoRActJAK2*p2EpoRpJAK2)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1)
    v7 = (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p1EpoRpJAK2
    v8 = (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p2EpoRpJAK2
    v9 = (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p12EpoRpJAK2
    v10 = EpoRJAK2_CIS*(EpoRCISRemove / init_EpoRJAK2)*(p12EpoRpJAK2 + p1EpoRpJAK2)
    v11 = SHP1*(SHP1ActEpoR / init_EpoRJAK2)*(EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2)
    v12 = SHP1Dea*SHP1Act
    v13 = (STAT5*(STAT5ActJAK2 / init_EpoRJAK2)*(EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2))/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1)
    v14 = (STAT5*(STAT5ActEpoR / init_EpoRJAK2^2)*(p12EpoRpJAK2 + p1EpoRpJAK2)^2)/((CIS*(CISInh / CISEqc) + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1))
    v15 = STAT5Imp*pSTAT5
    v16 = STAT5Exp*npSTAT5
    v17 = -(CISRNAEqc / init_STAT5)*CISRNATurn*npSTAT5*(ActD - 1)
    v18 = CISnRNA1*CISRNADelay
    v19 = CISnRNA2*CISRNADelay
    v20 = CISnRNA3*CISRNADelay
    v21 = CISnRNA4*CISRNADelay
    v22 = CISnRNA5*CISRNADelay
    v23 = CISRNA*CISRNATurn
    v24 = CISRNA*CISEqc*CISTurn
    v25 = CIS*CISTurn
    v26 = CISoe*CISTurn*CISEqcOE*CISEqc
    v27 = -(SOCS3RNAEqc / init_STAT5)*SOCS3RNATurn*npSTAT5*(ActD - 1)
    v28 = SOCS3nRNA1*SOCS3RNADelay
    v29 = SOCS3nRNA2*SOCS3RNADelay
    v30 = SOCS3nRNA3*SOCS3RNADelay
    v31 = SOCS3nRNA4*SOCS3RNADelay
    v32 = SOCS3nRNA5*SOCS3RNADelay
    v33 = SOCS3RNA*SOCS3RNATurn
    v34 = SOCS3RNA*SOCS3Eqc*SOCS3Turn
    v35 = SOCS3*SOCS3Turn
    v36 = SOCS3oe*SOCS3Turn*SOCS3EqcOE*SOCS3Eqc
    
    # rhs with parameters transformations
    du[1] = EpoRpJAK2*(JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act + (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p12EpoRpJAK2 + (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p1EpoRpJAK2 + (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p2EpoRpJAK2 - (Epo*EpoRJAK2*JAK2ActEpo)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1)
    du[2] = (Epo*EpoRJAK2*JAK2ActEpo)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1) - (EpoRpJAK2*EpoRActJAK2)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1) - (3*EpoRpJAK2*EpoRActJAK2)/((EpoRCISInh*EpoRJAK2_CIS + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1)) - EpoRpJAK2*(JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act
    du[3] = (EpoRpJAK2*EpoRActJAK2)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1) - (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p1EpoRpJAK2 - (3*EpoRActJAK2*p1EpoRpJAK2)/((EpoRCISInh*EpoRJAK2_CIS + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1))
    du[4] = (3*EpoRpJAK2*EpoRActJAK2)/((EpoRCISInh*EpoRJAK2_CIS + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1)) - (EpoRActJAK2*p2EpoRpJAK2)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1) - (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p2EpoRpJAK2
    du[5] = (EpoRActJAK2*p2EpoRpJAK2)/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1) - (JAK2EpoRDeaSHP1 / init_SHP1)*SHP1Act*p12EpoRpJAK2 + (3*EpoRActJAK2*p1EpoRpJAK2)/((EpoRCISInh*EpoRJAK2_CIS + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1))
    du[6] = -EpoRJAK2_CIS*(EpoRCISRemove / init_EpoRJAK2)*(p12EpoRpJAK2 + p1EpoRpJAK2)
    du[7] = SHP1Dea*SHP1Act - SHP1*(SHP1ActEpoR / init_EpoRJAK2)*(EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2)
    du[8] = SHP1*(SHP1ActEpoR / init_EpoRJAK2)*(EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) - SHP1Dea*SHP1Act
    du[9] = (STAT5Exp*npSTAT5*nuc)/cyt - (STAT5*(STAT5ActJAK2 / init_EpoRJAK2)*(EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2))/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1) - (STAT5*(STAT5ActEpoR / init_EpoRJAK2^2)*(p12EpoRpJAK2 + p1EpoRpJAK2)^2)/((CIS*(CISInh / CISEqc) + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1))
    du[10] = (STAT5*(STAT5ActJAK2 / init_EpoRJAK2)*(EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2))/(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1) - STAT5Imp*pSTAT5 + (STAT5*(STAT5ActEpoR / init_EpoRJAK2^2)*(p12EpoRpJAK2 + p1EpoRpJAK2)^2)/((CIS*(CISInh / CISEqc) + 1)*(SOCS3*(SOCS3Inh / SOCS3Eqc) + 1))
    du[11] = (STAT5Imp*cyt*pSTAT5)/nuc - STAT5Exp*npSTAT5
    du[12] = - CISnRNA1*CISRNADelay - (CISRNAEqc / init_STAT5)*CISRNATurn*npSTAT5*(ActD - 1)
    du[13] = CISnRNA1*CISRNADelay - CISnRNA2*CISRNADelay
    du[14] = CISnRNA2*CISRNADelay - CISnRNA3*CISRNADelay
    du[15] = CISnRNA3*CISRNADelay - CISnRNA4*CISRNADelay
    du[16] = CISnRNA4*CISRNADelay - CISnRNA5*CISRNADelay
    du[17] = (CISnRNA5*CISRNADelay*nuc)/cyt - CISRNA*CISRNATurn
    du[18] = CISRNA*CISEqc*CISTurn - CIS*CISTurn + CISoe*CISTurn*CISEqcOE * CISEqc
    du[19] = -SOCS3nRNA1*SOCS3RNADelay - (SOCS3RNAEqc / init_STAT5)*SOCS3RNATurn*npSTAT5*(ActD - 1)
    du[20] = SOCS3nRNA1*SOCS3RNADelay - SOCS3nRNA2*SOCS3RNADelay
    du[21] = SOCS3nRNA2*SOCS3RNADelay - SOCS3nRNA3*SOCS3RNADelay
    du[22] = SOCS3nRNA3*SOCS3RNADelay - SOCS3nRNA4*SOCS3RNADelay
    du[23] = SOCS3nRNA4*SOCS3RNADelay - SOCS3nRNA5*SOCS3RNADelay
    du[24] = (SOCS3nRNA5*SOCS3RNADelay*nuc)/cyt - SOCS3RNA*SOCS3RNATurn
    du[25] = SOCS3RNA*SOCS3Eqc*SOCS3Turn - SOCS3*SOCS3Turn + SOCS3oe*SOCS3Turn*SOCS3EqcOE * SOCS3Eqc
end;

The initial conditions and the ODE Problem 

In [2]:
# IVP
function u_init(p,t0)
    u0 = zeros(25)
    u0[1] = p[27]
    u0[7] = p[28]
    u0[9] = p[29]
    return u0
end

prob(p,epo_level,ActD,CISoe,SOCS3oe) = ODEProblem((du,u,p,t)->ode_func(du,u,p,t,epo_level,ActD,CISoe,SOCS3oe), u_init, (0.0,100.0), p);

### Likelihood Function

The likelihood function is defined as

In [3]:
function L(sim,data,σ)
    loss = 0.0
    obs = names(data)[2:end]  

    for (i,id) in enumerate(obs)
        loss_i = L_component(sim[i,:],data[!,id],σ[i])
        loss += loss_i
    end
    return loss
end
    
function L_component(sim,data,σ_i)
    loss_i = 0.0
    
    for i in eachindex(sim)
        if !isnan(data[i])
            loss_i += ((sim[i]-data[i])/σ_i)^2 + 2*log(sqrt(2π)*σ_i)
        end
    end
    return loss_i
end

L_component (generic function with 1 method)

### Experimental Data

In [4]:
data_CFUE_Long = DataFrame!(CSV.File("./JAK2STAT5_CFUE_Long_log10.csv"))
data_CFUE_Concentrations = DataFrame!(CSV.File("./JAK2STAT5_CFUE_Concentrations_log10.csv"))
data_CFUE_RNA = DataFrame!(CSV.File("./JAK2STAT5_CFUE_RNA_log10.csv"))
data_CFUE_ActD = [
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_ActD0_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_ActD1_log10.csv"))
]
data_CFUE_Fine = DataFrame!(CSV.File("./JAK2STAT5_CFUE_Fine_log10.csv"))
data_CFUE_CISoe =[
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_CISoe0_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_CISoe1_log10.csv"))
]
data_CFUE_CISoe_pEpoR =[
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_CISoe_pEpoR0_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_CISoe_pEpoR1_log10.csv"))
]
data_CFUE_SOCS3oe =[
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_SOCS3oe0_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_SOCS3oe1_log10.csv"))
]
data_CFUE_SHP1oe =[
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_SHP1oe0_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_SHP1oe1_log10.csv"))
]
data_CFUE_DR7 =[
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR72.5e-5_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR72.5e-6_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR72.5e-7_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR72.5e-8_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR72.5e-9_log10.csv"))
]
data_CFUE_DR30 =[
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR301.25e-6_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR301.25e-7_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR302.5e-6_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR302.5e-7_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR302.5e-8_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR302.5e-9_log10.csv"))
]
data_CFUE_DR10 =[
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR101.25e-7_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR101.25e-8_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR101.75e-8_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR101.7675e-7_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR102.5e-6_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR102.5e-7_log10.csv"))
]
data_CFUE_DR90 =[
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR901.25e-7_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR902.5e-6_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR902.5e-7_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR902.5e-8_log10.csv")),
    DataFrame!(CSV.File("./JAK2STAT5_CFUE_DR902.5e-9_log10.csv"))
];

UndefVarError: UndefVarError: `DataFrame!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
# fitted sd
const sd_CIS_abs = 0.071666890399901
const sd_CIS_au = 0.134037028009804
const sd_JAK2EpoR_au = 0.191838137085675
const sd_RNA_fold = 0.103074376969697
const sd_SHP1_abs = 0.064904672549311
const sd_SHP1_au = 0.077438644411215
const sd_SOCS3_abs = 0.10271619027042
const sd_SOCS3_au = 0.076348614779106
const sd_STAT5_abs = 0.119518171917816
const sd_STAT5_au = 0.123518909217589
const sd_pSTAT5_rel = 2.56704055700378
const sd_pSTAT5_socs3oe = 0.573852497761907;

In [6]:
function solve_prob(prob, saveat, saving_func)
    
    out = SavedValues(Float64, Vector{Float64})
    scb = SavingCallback(saving_func, out, saveat=saveat)
    
    tspan = (0.0, saveat[end])
    
    prob = remake(prob, tspan=tspan, callback=scb)
    
    sol = solve(prob, 
                CVODE_BDF();
                save_start = false,
                save_end = false,
                save_everystep = false,
                reltol = 1e-12,
                abstol = 1e-17
    )
    return VectorOfArray(sol.prob.kwargs[:callback].affect!.saved_values.saveval)
end

solve_prob (generic function with 1 method)

#### Experiment: Long time-course of JAK2-STAT5 phosphorylation dynamics in CFU-E cells (CFU-E Long)

In [7]:
# fitted offset and scale parameters
const offset_pJAK2_long = 0.009473351822798
const offset_pEpoR_long = 0.004391399791279
const offset_CIS_long = 0.026460000624405
const offset_SOCS3_long = 0.114475234936677
const offset_pSTAT5_long = 0.001093070638337
const scale_tSTAT5_long = 0.762143611844065
const scale_pJAK2_long = 0.882775356861714
const scale_pEpoR_long = 0.256529988627087
const scale_CIS_long = 16.4036102124066
const scale_SOCS3_long = 15.485524391452
const scale_pSTAT5_long = 1.47372211945355

function saving_CFUE_Long(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pJAK2_au = log10(offset_pJAK2_long + 2 * scale_pJAK2_long * (EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    pEpoR_au = log10(offset_pEpoR_long + 16 * scale_pEpoR_long * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    CIS_au = log10(offset_CIS_long + CIS * scale_CIS_long / CISEqc )  #??? / (CISEqc * CISRNAEqc * init_STAT5)
    SOCS3_au = log10(offset_SOCS3_long + SOCS3 * scale_SOCS3_long / SOCS3Eqc )  #??? / (SOCS3Eqc * SOCS3RNAEqc * init_STAT5)
    tSTAT5_au = log10(scale_tSTAT5_long * (STAT5 + pSTAT5) / init_STAT5)
    pSTAT5_au = log10(offset_pSTAT5_long + pSTAT5 * scale_pSTAT5_long / init_STAT5)
    
    return [pJAK2_au, pEpoR_au, CIS_au, SOCS3_au, tSTAT5_au, pSTAT5_au]
end

function L_CFUE_Long(p)
    ActD = 0.0
    CISoe = 0.0
    SOCS3oe = 0.0
    SHP1oe = 0.0
    epo_level = 1.25e-7
    
    prob_CFUE_Long = prob(p,epo_level,ActD,CISoe,SOCS3oe)  
    saveat = Float64.(data_CFUE_Long[!,:time])
    
    sim = solve_prob(prob_CFUE_Long, saveat, saving_CFUE_Long)
    
    σ = [
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
        sd_CIS_au,
        sd_SOCS3_au,
        sd_STAT5_au,
        sd_STAT5_au
    ]
    loss = L(sim, data_CFUE_Long, σ) 
    #@show loss
    return loss
end

L_CFUE_Long (generic function with 1 method)

#### Experiment: Absolute concentrations of proteins in CFU-E cells (CFU-E Concentrations)

In [8]:
# fitted offset and scale parameters
const offset_pSTAT5_conc = 0.236827475949589

function saving_CFUE_Concentrations(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    STAT5_abs = log10(STAT5)
    SHP1_abs = log10(SHP1 + SHP1Act)
    CIS_abs = log10(CIS)
    SOCS3_abs = log10(SOCS3)
    pSTAT5B_rel = offset_pSTAT5_conc + 100 * pSTAT5 / (STAT5 + pSTAT5)
    
    return [STAT5_abs, SHP1_abs, CIS_abs, SOCS3_abs, pSTAT5B_rel]
end

function L_CFUE_Concentrations(p)
    ActD = 0.0
    CISoe = 0.0
    SOCS3oe = 0.0
    SHP1oe = 0.0
    epo_level = 1.25e-7
    
    prob_CFUE_Concentrations = prob(p,epo_level,ActD,CISoe,SOCS3oe)  
    saveat = Float64.(data_CFUE_Concentrations[!,:time])
    
    sim = solve_prob(prob_CFUE_Concentrations, saveat, saving_CFUE_Concentrations)
    
    σ = [
        sd_STAT5_abs,
        sd_SHP1_abs,
        sd_CIS_abs,
        sd_SOCS3_abs,
        sd_pSTAT5_rel
    ]
    loss = L(sim, data_CFUE_Concentrations, σ)
    #@show loss
    return loss 
    
end

L_CFUE_Concentrations (generic function with 1 method)

#### Experiment: Time-course of CIS and SOCS3 mRNA expression (CFU-E RNA)

In [9]:
# fitted offset and scale parameters
const scale_SOCS3RNA_foldA = 56.9631412510002
const scale_SOCS3RNA_foldB = 49.074989211721
const scale_SOCS3RNA_foldC = 80.7316820439984
const scale_CISRNA_foldA = 33.2822368277369
const scale_CISRNA_foldB = 31.0176342778512
const scale_CISRNA_foldC = 19.6194113406379

function saving_CFUE_RNA(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    SOCS3RNA_foldA = log10(SOCS3RNA * scale_SOCS3RNA_foldA / SOCS3RNAEqc + 1)
    SOCS3RNA_foldB = log10(SOCS3RNA * scale_SOCS3RNA_foldB / SOCS3RNAEqc + 1)
    SOCS3RNA_foldC = log10(SOCS3RNA * scale_SOCS3RNA_foldC / SOCS3RNAEqc + 1)
    CISRNA_foldA = log10(CISRNA * scale_CISRNA_foldA / CISRNAEqc + 1)
    CISRNA_foldB = log10(CISRNA * scale_CISRNA_foldB / CISRNAEqc + 1)
    CISRNA_foldC = log10(CISRNA * scale_CISRNA_foldC / CISRNAEqc + 1)
    
    return [SOCS3RNA_foldA, SOCS3RNA_foldB, SOCS3RNA_foldC, CISRNA_foldA, CISRNA_foldB, CISRNA_foldC]
end

function L_CFUE_RNA(p)
    ActD = 0.0
    CISoe = 0.0
    SOCS3oe = 0.0
    SHP1oe = 0.0
    epo_level = 1.25e-7
    
    prob_CFUE_RNA = prob(p,epo_level,ActD,CISoe,SOCS3oe)  
    saveat = Float64.(data_CFUE_RNA[!,:time])
    
    sim = solve_prob(prob_CFUE_RNA, saveat, saving_CFUE_RNA)
    
    σ = [
        sd_RNA_fold,
        sd_RNA_fold,
        sd_RNA_fold,
        sd_RNA_fold,
        sd_RNA_fold,
        sd_RNA_fold
    ]
    loss = L(sim, data_CFUE_RNA, σ)
    #@show loss
    return loss
    
end

L_CFUE_RNA (generic function with 1 method)

#### Experiment: Time-course of JAK2-STAT5 phosphorylation dynamics in CFU-E cells with Actinomycin D treatment (CFU-E ActD)

In [10]:
# fitted offset and scale parameters
const offset_pJAK2_actd = 0.017059890602388
const offset_pEpoR_actd = 0.018840236081137
const offset_CIS_actd = 0.009388328725541
const offset_pSTAT5_actd = 0.001868785567417
const scale_tSTAT5_actd = 0.815207201779241
const scale_pJAK2_actd = 0.809491692283694
const scale_pEpoR_actd = 0.215657294488182
const scale_CIS_actd = 14.5490143154214
const scale_pSTAT5_actd = 1.09743435047828

function saving_CFUE_ActD(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pJAK2_au = log10(offset_pJAK2_actd + 2 * scale_pJAK2_actd * (EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    pEpoR_au = log10(offset_pEpoR_actd + 16 * scale_pEpoR_actd * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    tSTAT5_au = log10(scale_tSTAT5_actd * (STAT5 + pSTAT5) / init_STAT5)
    pSTAT5_au = log10(offset_pSTAT5_actd + pSTAT5 * scale_pSTAT5_actd / init_STAT5)
    CIS_au = log10(offset_CIS_actd + CIS * scale_CIS_actd / CISEqc )  #??? / (CISEqc * CISRNAEqc * init_STAT5)
    
    return [pJAK2_au, pEpoR_au, tSTAT5_au, pSTAT5_au, CIS_au]
end

function L_CFUE_ActD(p)

    CISoe = 0.0
    SOCS3oe = 0.0
    SHP1oe = 0.0
    epo_level = 1.25e-7
    
    σ = [
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
        sd_STAT5_au,
        sd_STAT5_au,
        sd_CIS_au
    ]
    
    loss = 0.0
    for (i,ActD) in enumerate([0.0, 1.0])
    
        prob_CFUE_ActD = prob(p,epo_level,ActD,CISoe,SOCS3oe)  
        saveat = Float64.(data_CFUE_ActD[i][!,:time])
    
        sim = solve_prob(prob_CFUE_ActD, saveat, saving_CFUE_ActD)
        
        loss += L(sim, data_CFUE_ActD[i], σ) 
    end
    #@show loss
    return loss 
end

L_CFUE_ActD (generic function with 1 method)

#### Experiment: Time-course of JAK2-STAT5 phosphorylation dynamics in CFU-E cells densely sampled (CFU-E Fine)

In [11]:
# fitted offset and scale parameters
const offset_pJAK2_fine = 0.021658276199496
const offset_pEpoR_fine = 0.064550876993068
const scale_pJAK2_fine = 0.400770707451671
const scale_pEpoR_fine = 0.080269467995686

function saving_CFUE_Fine(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pJAK2_au = log10(offset_pJAK2_fine + 2 * scale_pJAK2_fine * (EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    pEpoR_au = log10(offset_pEpoR_fine + 16 * scale_pEpoR_fine * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)

    return [pJAK2_au, pEpoR_au]
end

function L_CFUE_Fine(p)
    ActD = 0.0
    CISoe = 0.0
    SOCS3oe = 0.0
    SHP1oe = 0.0
    epo_level = 1.25e-6
    
    prob_CFUE_Fine = prob(p,epo_level,ActD,CISoe,SOCS3oe)  
    saveat = Float64.(data_CFUE_Fine[!,:time])
    
    sim = solve_prob(prob_CFUE_Fine, saveat, saving_CFUE_Fine)
    
    σ = [
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
    ]
    loss = L(sim, data_CFUE_Fine, σ) 
    #@show loss
    return loss
    
end

L_CFUE_Fine (generic function with 1 method)

#### Experiment: Time-course of JAK2-STAT5 phosphorylation dynamics in CFU-E cells over-expressing CIS (CFU-E CISoe)

In [12]:
# fitted offset and scale parameters
const offset_pJAK2_cisoe = 0.021928076840931
const offset_pEpoR_cisoe = 0.030210854745913
const offset_CIS_cisoe = 0.030012199170223
const offset_SOCS3_cisoe = 0.278270301775932
const offset_pSTAT5_cisoe = 0.070791697579138
const scale_pJAK2_cisoe = 1.85429884799069
const scale_pEpoR_cisoe = 0.273625536372832
const scale_CIS_cisoe = 1.37088201048354
const scale_SOCS3_cisoe = 11.5682149955845
const scale_pSTAT5_cisoe = 2.41904111907135

function saving_CFUE_CISoe(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pJAK2_au = log10(offset_pJAK2_cisoe + 2 * scale_pJAK2_cisoe * (EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    pEpoR_au = log10(offset_pEpoR_cisoe + 16 * scale_pEpoR_cisoe * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    CIS_au = log10(offset_CIS_cisoe + CIS * scale_CIS_cisoe / CISEqc )  #??? / (CISEqc * CISRNAEqc * init_STAT5)
    SOCS3_au = log10(offset_SOCS3_cisoe + SOCS3 * scale_SOCS3_cisoe / SOCS3Eqc )  #??? / (SOCS3Eqc * SOCS3RNAEqc * init_STAT5)
    pSTAT5_au = log10(offset_pSTAT5_cisoe + pSTAT5 * scale_pSTAT5_cisoe / init_STAT5)
    
    return [pJAK2_au, pEpoR_au, CIS_au, SOCS3_au, pSTAT5_au]
end

function u_init_CISoe(p,t0)
    u0 = zeros(25)
    u0[1] = p[27]
    u0[7] = p[28]
    u0[9] = p[29]
    u0[18] = p[1]*p[2]
    u0[6] = 1.0
    return u0
end

function L_CFUE_CISoe(p)
    ActD = 0.0
    SOCS3oe = 0.0
    SHP1oe = 0.0
    epo_level = 1.25e-7
    
    σ = [
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
        sd_CIS_au,
        sd_SOCS3_au,
        sd_STAT5_au
    ]
    
    loss = 0.0
    for (i,CISoe) in enumerate([0.0, 1.0]) 
        prob_CFUE_CISoe = prob(p,epo_level,ActD,CISoe,SOCS3oe)
        if CISoe == 1.0
           prob_CFUE_CISoe = remake(prob_CFUE_CISoe, u0=u_init_CISoe)
        end
        saveat = Float64.(data_CFUE_CISoe[i][!,:time])

        sim = solve_prob(prob_CFUE_CISoe, saveat, saving_CFUE_CISoe)
        loss += L(sim, data_CFUE_CISoe[i], σ)
    end
    #@show loss
    return loss
end

L_CFUE_CISoe (generic function with 1 method)

#### Experiment: Time-course of EpoR phosphorylation dynamics in CFU-E cells over-expressing CIS (CFU-E CISoe)

In [13]:
# fitted offset and scale parameters
const offset_pEpoR_cisoe_pepor = 0.131027220027866
const scale_pEpoR_cisoe_pepor = 0.156616708581292

function saving_CFUE_CISoe_pEpoR(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pEpoR_au = log10(offset_pEpoR_cisoe_pepor + 16 * scale_pEpoR_cisoe_pepor * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    
    return [pEpoR_au]
end

function u_init_CISoe_pEpoR(p,t0)
    u0 = zeros(25)
    u0[1] = p[27]
    u0[7] = p[28]
    u0[9] = p[29]
    u0[18] = p[1]*p[2]
    u0[6] = 1.0
    return u0
end

function L_CFUE_CISoe_pEpoR(p)
    ActD = 0.0
    SOCS3oe = 0.0
    SHP1oe = 0.0
    epo_level = 1.25e-7
    
    σ = [
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
        sd_CIS_au,
        sd_SOCS3_au,
        sd_STAT5_au
    ]
    
    loss = 0.0
    for (i,CISoe) in enumerate([0.0, 1.0]) 
        prob_CFUE_CISoe_pEpoR = prob(p,epo_level,ActD,CISoe,SOCS3oe)
        if CISoe == 1.0
           prob_CFUE_CISoe_pEpoR = remake(prob_CFUE_CISoe_pEpoR, u0=u_init_CISoe_pEpoR)
        end
        saveat = Float64.(data_CFUE_CISoe_pEpoR[i][!,:time])

        sim = solve_prob(prob_CFUE_CISoe_pEpoR, saveat, saving_CFUE_CISoe_pEpoR)
        loss += L(sim, data_CFUE_CISoe_pEpoR[i], σ)
    end
    #@show loss
    return loss
end

L_CFUE_CISoe_pEpoR (generic function with 1 method)

#### Experiment: Time-course of JAK2-STAT5 phosphorylation dynamics in CFU-E cells over-expressing SOCS3 (CFU-E SOCS3oe)

In [14]:
# fitted offset and scale parameters
const offset_pJAK2_socs3oe = 0.059752453075581
const offset_pEpoR_socs3oe = 0.056711915845858
const offset_CIS_socs3oe = 0.090496783983191
const offset_SOCS3_socs3oe = 0.025584406819207
const offset_pSTAT5_socs3oe = 0.005613364786273
const scale_pJAK2_socs3oe =  1.266550022901
const scale_pEpoR_socs3oe = 0.637275573973593
const scale_CIS_socs3oe = 21.1493834462165
const scale_SOCS3_socs3oe = 1.20457246908681
const scale_pSTAT5_socs3oe = 1.76032560825919

function saving_CFUE_SOCS3oe(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pJAK2_au = log10(offset_pJAK2_socs3oe + 2 * scale_pJAK2_socs3oe * (EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    pEpoR_au = log10(offset_pEpoR_socs3oe + 16 * scale_pEpoR_socs3oe * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    CIS_au = log10(offset_CIS_socs3oe + CIS * scale_CIS_socs3oe / CISEqc )  #??? / (CISEqc * CISRNAEqc * init_STAT5)
    SOCS3_au = log10(offset_SOCS3_socs3oe + SOCS3 * scale_SOCS3_socs3oe / SOCS3Eqc )  #??? / (SOCS3Eqc * SOCS3RNAEqc * init_STAT5)
    pSTAT5_au = log10(offset_pSTAT5_socs3oe + pSTAT5 * scale_pSTAT5_socs3oe / init_STAT5)
    
    return [pJAK2_au, pEpoR_au, CIS_au, SOCS3_au, pSTAT5_au]
end

function u_init_SOCS3oe(p,t0)
    u0 = zeros(25)
    u0[1] = p[27]
    u0[7] = p[28]
    u0[9] = p[29]
    u0[25] = p[16]*p[17]
    return u0
end

function L_CFUE_SOCS3oe(p)
    ActD = 0.0
    CISoe = 0.0
    SHP1oe = 0.0
    epo_level = 1.25e-7
    
    loss = 0.0
    for (i,SOCS3oe) in enumerate([0.0, 1.0]) 
        σ = [
            sd_JAK2EpoR_au,
            sd_JAK2EpoR_au,
            sd_CIS_au,
            sd_SOCS3_au,
            sd_STAT5_au + SOCS3oe * sd_pSTAT5_socs3oe
        ]
        
        prob_CFUE_SOCS3oe = prob(p,epo_level,ActD,CISoe,SOCS3oe)  
        
        if SOCS3oe == 1.0
           prob_CFUE_SOCS3oe = remake(prob_CFUE_SOCS3oe, u0=u_init_SOCS3oe)
        end
        saveat = Float64.(data_CFUE_SOCS3oe[i][!,:time])
    
        sim = solve_prob(prob_CFUE_SOCS3oe, saveat, saving_CFUE_SOCS3oe)
        
        loss += L(sim, data_CFUE_SOCS3oe[i], σ) 
    end
    #@show loss
    return loss 
end

L_CFUE_SOCS3oe (generic function with 1 method)

#### Experiment: Time-course of JAK2-STAT5 phosphorylation dynamics in CFU-E cells over-expressing SHP1 (CFU-E SHP1oe)

In [15]:
# fitted offset and scale parameters
const offset_pJAK2_shp1oe = 0.026667365128327
const offset_pEpoR_shp1oe = 0.031327226759826
const offset_CIS_shp1oe = 0.058794513917949
const offset_pSTAT5_shp1oe = 0.054941495350482
const scale_tSTAT5_shp1oe = 0.681101374792296
const scale_pJAK2_shp1oe =  2.31011130113927
const scale_pEpoR_shp1oe = 0.241283495329875
const scale_CIS_shp1oe = 51.7115711834136
const scale_SHP1_shp1oe = 0.225547534442942
const scale_pSTAT5_shp1oe = 1.15158552971751

function saving_CFUE_SHP1oe(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pJAK2_au = log10(offset_pJAK2_shp1oe + 2 * scale_pJAK2_shp1oe * (EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    pEpoR_au = log10(offset_pEpoR_shp1oe + 16 * scale_pEpoR_shp1oe * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    CIS_au = log10(offset_CIS_shp1oe + CIS * scale_CIS_shp1oe / CISEqc)   #??? / (CISEqc * CISRNAEqc * init_STAT5)
    tSTAT5_au = log10(scale_tSTAT5_shp1oe * (STAT5 + pSTAT5) / init_STAT5)
    pSTAT5_au = log10(offset_pSTAT5_shp1oe + pSTAT5 * scale_pSTAT5_shp1oe / init_STAT5)
    tSHP1_au = log10(scale_SHP1_shp1oe * (SHP1 + SHP1Act) / init_SHP1) # *(SHP1oe * SHP1ProOE + 1)
    
    return [pJAK2_au, pEpoR_au, CIS_au, tSTAT5_au, pSTAT5_au, tSHP1_au]
end

function u_init_SHP1oe(p,t0)
    u0 = zeros(25)
    u0[1] = p[27]
    u0[7] = p[28]*(p[15]+1)
    u0[9] = p[29]
    return u0
end

function L_CFUE_SHP1oe(p)
    ActD = 0.0
    CISoe = 0.0
    SOCS3oe = 0.0
    epo_level = 1.25e-7
    
    σ = [
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
        sd_CIS_au,
        sd_STAT5_au,
        sd_STAT5_au,
        sd_SHP1_au
    ]
    
    prob_CFUE_SHP1oe = prob(p,epo_level,ActD,CISoe,SOCS3oe) 
    
    loss = 0.0
    for (i,SHP1oe) in enumerate([0.0, 1.0]) 
        
        if SHP1oe == 1.0
           prob_CFUE_SHP1oe = remake(prob_CFUE_SHP1oe, u0=u_init_SHP1oe)
        end
        saveat = Float64.(data_CFUE_SHP1oe[i][!,:time])
    
        sim = solve_prob(prob_CFUE_SHP1oe, saveat, saving_CFUE_SHP1oe)
        loss += L(sim, data_CFUE_SHP1oe[i], σ) 
    end
    #@show loss
    return loss 
end

L_CFUE_SHP1oe (generic function with 1 method)

#### Experiment: Dose response of JAK2 and EpoR phosphorylation in CFU-E cells 7 minutes after Epo stimulation (CFU-E DoseResp 7min)

In [16]:
# fitted offset and scale parameters
const offset_pJAK2_dr7 = 0.04733977489635
const offset_pEpoR_dr7 = 0.028405110847623
const scale_pJAK2_dr7 =  0.506322186585869
const scale_pEpoR_dr7 = 0.101429038197386


function saving_CFUE_DR7(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pJAK2_au = log10(offset_pJAK2_dr7 + 2 * scale_pJAK2_dr7 * (EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    pEpoR_au = log10(offset_pEpoR_dr7 + 16 * scale_pEpoR_dr7 * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)

    return [pJAK2_au, pEpoR_au]
end


function L_CFUE_DR7(p)
    ActD = 0.0
    CISoe = 0.0
    SHP1oe = 0.0
    SOCS3oe = 0.0
    
    σ = [
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
    ]
    
    loss = 0.0
    for (i,epo_level) in enumerate([2.5E-05, 2.5E-06, 2.5E-07, 2.5E-08, 2.5E-09]) 
        prob_CFUE_DR7 = prob(p,epo_level,ActD,CISoe,SOCS3oe)
        saveat = Float64.(data_CFUE_DR7[i][!,:time])
    
        sim = solve_prob(prob_CFUE_DR7, saveat, saving_CFUE_DR7)
        loss += L(sim, data_CFUE_DR7[i], σ) 
    end
    #@show loss
    return loss 
end

L_CFUE_DR7 (generic function with 1 method)

#### Experiment: Dose response of JAK2 and EpoR phosphorylation in CFU-E cells 30 minutes after Epo stimulation (CFU-E DoseResp 30min)

In [17]:
# fitted offset and scale parameters
const offset_pJAK2_dr30 = 0.029944254724382
const offset_pEpoR_dr30 = 0.001
const scale_pJAK2_dr30 =  1.77561150477407
const scale_pEpoR_dr30 = 0.537385100441811


function saving_CFUE_DR30(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pJAK2_au = log10(offset_pJAK2_dr30 + 2 * scale_pJAK2_dr30 * (EpoRpJAK2 + p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)
    pEpoR_au = log10(offset_pEpoR_dr30 + 16 * scale_pEpoR_dr30 * (p12EpoRpJAK2 + p1EpoRpJAK2 + p2EpoRpJAK2) / init_EpoRJAK2)

    return [pJAK2_au, pEpoR_au]
end


function L_CFUE_DR30(p)
    ActD = 0.0
    CISoe = 0.0
    SHP1oe = 0.0
    SOCS3oe = 0.0
    
    σ = [
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
    ]
    
    loss = 0.0
    for (i,epo_level) in enumerate([1.25E-06, 1.25E-07, 2.5E-06, 2.5E-07, 2.5E-08, 2.5E-09]) 
        prob_CFUE_DR30 = prob(p,epo_level,ActD,CISoe,SOCS3oe)
        saveat = Float64.(data_CFUE_DR30[i][!,:time])
    
        sim = solve_prob(prob_CFUE_DR30, saveat, saving_CFUE_DR30)
        loss += L(sim, data_CFUE_DR30[i], σ) 
    end
    #@show loss
    return loss 
end

L_CFUE_DR30 (generic function with 1 method)

#### Experiment: Dose response of STAT5 phosphorylation in CFU-E cells 10 minutes after Epo stimulation (CFU-E DoseResp pSTAT5 10min)

In [18]:
# fitted offset and scale parameters
const scale_pSTAT_dr10 =  1.00950903665707

function saving_CFUE_DR10(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    pSTAT5_au = log10(pSTAT5 * scale_pSTAT_dr10 / init_STAT5)
    return [pSTAT5_au]
end


function L_CFUE_DR10(p)
    ActD = 0.0
    CISoe = 0.0
    SHP1oe = 0.0
    SOCS3oe = 0.0
    
    σ = [
        sd_STAT5_au
    ]
    
    loss = 0.0
    for (i,epo_level) in enumerate([1.25E-07, 1.25E-08, 2.5E-06, 2.5E-07, 2.5E-08, 2.5E-09]) 
        prob_CFUE_DR10 = prob(p,epo_level,ActD,CISoe,SOCS3oe)
        saveat = Float64.(data_CFUE_DR10[i][!,:time])
    
        sim = solve_prob(prob_CFUE_DR10, saveat, saving_CFUE_DR10)
        loss += L(sim, data_CFUE_DR10[i], σ) 
    end
    #@show loss
    return loss 
end

L_CFUE_DR10 (generic function with 1 method)

#### Experiment: Dose response of CIS expression in CFU-E cells 90 minutes after Epo stimulation (CFU-E DoseResp CIS 90min)

In [19]:
# fitted offset and scale parameters
const scale1_CIS_dr90 = 17.5631450183542
const scale2_CIS_dr90 = 16.1205634424041

function saving_CFUE_DR90(u, t, integrator)
    (EpoRJAK2,EpoRpJAK2,p1EpoRpJAK2,p2EpoRpJAK2,p12EpoRpJAK2,EpoRJAK2_CIS,SHP1,SHP1Act,STAT5,pSTAT5,npSTAT5,
    CISnRNA1,CISnRNA2,CISnRNA3,CISnRNA4,CISnRNA5,CISRNA,CIS,SOCS3nRNA1,SOCS3nRNA2,SOCS3nRNA3,SOCS3nRNA4,
    SOCS3nRNA5,SOCS3RNA,SOCS3) = u
    (CISEqc,CISEqcOE,CISInh,CISRNADelay,CISRNAEqc,CISRNATurn,CISTurn,EpoRActJAK2,EpoRCISInh,
    EpoRCISRemove,JAK2ActEpo,JAK2EpoRDeaSHP1,SHP1ActEpoR,SHP1Dea,SHP1ProOE,SOCS3Eqc,SOCS3EqcOE,
    SOCS3Inh,SOCS3RNADelay,SOCS3RNAEqc,SOCS3RNATurn,SOCS3Turn,STAT5ActEpoR,STAT5ActJAK2,STAT5Exp,
    STAT5Imp,init_EpoRJAK2,init_SHP1,init_STAT5) = integrator.p
    
    # output values
    CIS_au1 = log10(CIS * scale1_CIS_dr90 / CISEqc)
    CIS_au2 = log10(CIS * scale2_CIS_dr90 / CISEqc)
    
    return [CIS_au1, CIS_au2]
end


function L_CFUE_DR90(p)
    ActD = 0.0
    CISoe = 0.0
    SHP1oe = 0.0
    SOCS3oe = 0.0
    
    σ = [
        sd_CIS_au,
        sd_CIS_au
    ]
    
    loss = 0.0
    for (i,epo_level) in enumerate([1.25E-07, 2.5E-06, 2.5E-07, 2.5E-08, 2.5E-09]) 
        prob_CFUE_DR90 = prob(p,epo_level,ActD,CISoe,SOCS3oe)
        saveat = Float64.(data_CFUE_DR90[i][!,:time])
    
        sim = solve_prob(prob_CFUE_DR90, saveat, saving_CFUE_DR90)
        loss += L(sim, data_CFUE_DR90[i], σ) 
    end
    #@show loss
    return loss 
end

L_CFUE_DR90 (generic function with 1 method)

### Best Fit Parameters

Best fit parameters reported in the article

In [20]:
const p_best = (
    CISEqc = 432.854027325732,
    CISEqcOE = 0.53027229006794,
    CISInh = 785221045.232906,
    CISRNADelay = 0.144777009620017,
    CISRNAEqc = 1.0,
    CISRNATurn = 1000.0,
    CISTurn = 0.008398825029262,
    EpoRActJAK2 = 0.267299659481333,
    EpoRCISInh = 1000000.0,
    EpoRCISRemove = 5.42989282766407,
    JAK2ActEpo = 633154.209632843,
    JAK2EpoRDeaSHP1 = 142.723069412318,
    SHP1ActEpoR = 0.001,
    SHP1Dea = 0.008162255244283,
    SHP1ProOE = 2.82568032716189,
    SOCS3Eqc = 173.64095522801,
    SOCS3EqcOE = 0.679185853534469,
    SOCS3Inh = 10.4074255973465,
    SOCS3RNADelay = 1.06454014521077,
    SOCS3RNAEqc = 1.0,
    SOCS3RNATurn = 0.008309263619427,
    SOCS3Turn = 10000.0,
    STAT5ActEpoR = 38.9972748894298,
    STAT5ActJAK2 = 0.078108833252848,
    STAT5Exp = 0.074514698833232,
    STAT5Imp = 0.026887318057561,
    init_EpoRJAK2 = 3.97622379188569,
    init_SHP1 = 26.7251164163486,
    init_STAT5 = 79.753639977851
);

Total loss values is the sum of all losses

In [21]:
function L_TOT(params)
    
    p = [p_i for p_i in merge(p_best,params)]
    
    L_CFUE_Long(p) +
    L_CFUE_Concentrations(p) +
    L_CFUE_RNA(p) +
    L_CFUE_ActD(p) +
    L_CFUE_Fine(p) +
    L_CFUE_CISoe(p) +
    L_CFUE_CISoe_pEpoR(p) +
    L_CFUE_SOCS3oe(p) +
    L_CFUE_SHP1oe(p) +
    L_CFUE_DR7(p) +
    L_CFUE_DR30(p) +
    L_CFUE_DR10(p) +
    L_CFUE_DR90(p)
end

L_TOT (generic function with 1 method)

In [22]:
L_TOT(p_best)

UndefVarError: UndefVarError: `data_CFUE_Long` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

The value reported in the article is $$-2log(L) = -478.46$$

### Identifiability and confidence intervals of the estimated model parameters

In [23]:
p_init = (
    CISEqc = 432.854027325732,
    CISEqcOE = 0.53027229006794,
    CISInh = 785221045.232906,
    CISRNADelay = 0.144777009620017,
    #CISRNAEqc = 1.0,
    CISRNATurn = 1000.0,
    CISTurn = 0.008398825029262,
    EpoRActJAK2 = 0.267299659481333,
    EpoRCISInh = 1000000.0,
    EpoRCISRemove = 5.42989282766407,
    JAK2ActEpo = 633154.209632843,
    JAK2EpoRDeaSHP1 = 142.723069412318,
    SHP1ActEpoR = 0.001,
    SHP1Dea = 0.008162255244283,
    SHP1ProOE = 2.82568032716189,
    SOCS3Eqc = 173.64095522801,
    SOCS3EqcOE = 0.679185853534469,
    SOCS3Inh = 10.4074255973465,
    SOCS3RNADelay = 1.06454014521077,
    #SOCS3RNAEqc = 1.0,
    SOCS3RNATurn = 0.008309263619427,
    SOCS3Turn = 10000.0,
    STAT5ActEpoR = 38.9972748894298,
    STAT5ActJAK2 = 0.078108833252848,
    STAT5Exp = 0.074514698833232,
    STAT5Imp = 0.026887318057561,
    init_EpoRJAK2 = 3.97622379188569,
    init_SHP1 = 26.7251164163486,
    init_STAT5 = 79.753639977851
)

p_scan = (
    CISEqc = (1e-3, 1e4),
    CISEqcOE = (1e-3, 1e3),
    CISInh = (1e-3, 1e10),
    CISRNADelay = (1e-3, 1e3),
    #CISRNAEqc = 1.0,
    CISRNATurn = (1e-3, 1e4),
    CISTurn = (1e-3, 1e3),
    EpoRActJAK2 = (1e-3, 1e4),
    EpoRCISInh = (1e-3, 1e7),
    EpoRCISRemove = (1e-3, 1e3),
    JAK2ActEpo = (1e-3, 1e9),
    JAK2EpoRDeaSHP1 = (1e-3,1e10),
    SHP1ActEpoR = (1e-4,1e3),
    SHP1Dea = (1e-3,1e3),
    SHP1ProOE = (1e-3,1e3),
    SOCS3Eqc = (1e-3,1e3),
    SOCS3EqcOE = (1e-3,1e3),
    SOCS3Inh = (1e-3,1e3),
    SOCS3RNADelay = (1e-3,1e3),
    #SOCS3RNAEqc = 1.0,
    SOCS3RNATurn = (1e-3,1e3),
    SOCS3Turn = (1e-3,1e5),
    STAT5ActEpoR = (1e-3,1e3),
    STAT5ActJAK2 = (1e-3,1e3),
    STAT5Exp = (1e-3,1e3),
    STAT5Imp = (1e-3,1e3),
    init_EpoRJAK2 = (1e-3,1e3),
    init_SHP1 = (1e-3,1e3),
    init_STAT5 = (1e-3,1e3)
);

In [24]:
using CICOBase

α = L_TOT(p_best) + 3.84

intervals = Vector{ParamInterval}(undef,length(p_init))
for i in 1:length(p_init)
    intervals[i] = get_interval(
        [p for p in p_init],
        i,
        (p)->L_TOT(NamedTuple{keys(p_init)}(p)),
        :CICO_ONE_PASS,
        loss_crit = α,
        theta_bounds = [(pb[1]/2,pb[2]*2) for pb in p_scan],
        scan_bounds = p_scan[i],
        local_alg = :LN_NELDERMEAD,
        scale = fill(:log,length(p_init))
    )
    println(intervals[i]) 
end

UndefVarError: UndefVarError: `data_CFUE_Long` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [25]:
L(saveval, data_CFUE_Long,[
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
        sd_CIS_au,
        sd_SOCS3_au,
        sd_STAT5_au,
        sd_STAT5_au
    ] )

UndefVarError: UndefVarError: `saveval` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [26]:
1/(sqrt(2π)*prod([
        sd_JAK2EpoR_au,
        sd_JAK2EpoR_au,
        sd_CIS_au,
        sd_SOCS3_au,
        sd_STAT5_au,
        sd_STAT5_au
    ]))

69430.02502971576

In [27]:
df = DataFrame([recode(col, "NaN"=>missing) for col = eachcol(data_CFUE_Long)], names(data_CFUE_Long))

UndefVarError: UndefVarError: `data_CFUE_Long` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [28]:
DataFrame([recode(col, missing=>NaN) for col = eachcol(data_CFUE_Long)], names(data_CFUE_Long))


UndefVarError: UndefVarError: `data_CFUE_Long` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [29]:
#CSV.write("C:\\Julia-1.4.2\\dev\\likelihoodprofiler-cases\\notebook\\JAK2STAT5_CFUE_Long_log10.csv",df)

In [30]:
data_CFUE_DR7

UndefVarError: UndefVarError: `data_CFUE_DR7` not defined in `Main`
Suggestion: check for spelling errors or missing imports.